# Evaluation of the interval matrix exponential using Horner's method

Reference:

- https://www.mat.univie.ac.at/~neum/ms/intExp.pdf

In [88]:
using Revise, IntervalMatrices
using IntervalMatrices: _exp_remainder, _exp_remainder_series, scale_and_square

using LinearAlgebra
using LinearAlgebra: checksquare

In [67]:
# evaluate 
function H̃(K::Int, A::IntervalMatrix{T}) where {T}
    @assert K >= 1
    n = checksquare(A)
    Iₙ = IntervalMatrix(Interval(1.0) * I, n)
    out = Iₙ + A/K
    while K > 1
        K -= 1
        out = Iₙ + A/K * out
    end
    return out
end

# remainder, see eq. (4.4) in the paper
function R(K::Int, A::IntervalMatrix{T}) where {T}
    return _exp_remainder(A, 1.0, K)
    # NOTE: in the paper they use
    #_exp_remainder_series(A, 1.0, K)
    # but the former is more accurate
end

# cf. eq. (4.9) 
function H(K, A)
    H̃(K, A) + R(K,A)
end

H (generic function with 1 method)

In [68]:
function S(L, K, A)
    # scaling
    α = Interval(1/2)^L
    As = A * α
        
    # taylor evaluation with remainder using Horner's method
    T = H(K, As)

    # squaring
    out = copy(T)
    for i in 1:L
        out = square(out)
    end
    return out
end

S (generic function with 1 method)

## Test case

In [13]:
function tline(;η=2, R₀=1.00, Rd₀=10.0, L₀=1e-10, C₀=1e-13 * 4.00, scale=1e-9)
    A₁₁ = zeros(η, η)
    A₁₂ = Bidiagonal(fill(-1/C₀, η), fill(1/C₀, η-1), :U)
    A₂₁ = Bidiagonal(fill(1/L₀, η), fill(-1/L₀, η-1), :L)
    A₂₂ = Diagonal(vcat(-Rd₀/L₀, fill(-R₀/L₀, η-1)))
    A  = [A₁₁ A₁₂; A₂₁ A₂₂] .* scale
    return A
end

tline (generic function with 1 method)

In [73]:
S(10, 10, A)

4×4 IntervalMatrix{Float64,Interval{Float64},Array{Interval{Float64},2}}:
  [5.51581e-08, 5.51582e-08]   …   [4.02887e-07, 4.02888e-07] 
 [-3.80178e-08, -3.80177e-08]     [-1.68779e-07, -1.68778e-07]
  [1.70663e-09, 1.70664e-09]      [-2.86534e-08, -2.86533e-08]
 [-1.61155e-09, -1.61154e-09]      [6.29109e-08, 6.2911e-08]  

In [74]:
exp_overapproximation(A, 1.0, 10)

4×4 IntervalMatrix{Float64,Interval{Float64},Array{Interval{Float64},2}}:
 [-8.52379e+120, 8.52379e+120]  …  [-1.18779e+122, 1.18779e+122]
 [-8.52379e+120, 8.52379e+120]     [-1.18779e+122, 1.18779e+122]
 [-8.52379e+120, 8.52379e+120]     [-1.18779e+122, 1.18779e+122]
 [-8.52379e+120, 8.52379e+120]     [-1.18779e+122, 1.18779e+122]

In [59]:
A = IntervalMatrix(Matrix(tline(η=2)))

4×4 IntervalMatrix{Float64,Interval{Float64},Array{Interval{Float64},2}}:
     [0, 0]    [0, 0]  [-2500, -2500]    [2500, 2500]
     [0, 0]    [0, 0]          [0, 0]  [-2500, -2500]
   [10, 10]    [0, 0]    [-100, -100]          [0, 0]
 [-10, -10]  [10, 10]          [0, 0]      [-10, -10]

Note that the exp remainder of `A` bloats up; we *need* to do the scaling and squaring. 

In [75]:
_exp_remainder(A / 1000, 1.0, 10)

4×4 IntervalMatrix{Float64,Interval{Float64},Array{Interval{Float64},2}}:
 [-9.82548e-15, 9.82548e-15]  …  [-2.01483e-13, 2.01483e-13]
 [-9.82548e-15, 9.82548e-15]     [-2.01483e-13, 2.01483e-13]
 [-9.82548e-15, 9.82548e-15]     [-2.01483e-13, 2.01483e-13]
 [-9.82548e-15, 9.82548e-15]     [-2.01483e-13, 2.01483e-13]

In [76]:
_exp_remainder_series(A / 1000, 1.0, 10)

4×4 IntervalMatrix{Float64,Interval{Float64},Array{Interval{Float64},2}}:
 [-2.097, 2.097]  [-2.097, 2.097]  [-2.097, 2.097]  [-2.097, 2.097]
 [-2.097, 2.097]  [-2.097, 2.097]  [-2.097, 2.097]  [-2.097, 2.097]
 [-2.097, 2.097]  [-2.097, 2.097]  [-2.097, 2.097]  [-2.097, 2.097]
 [-2.097, 2.097]  [-2.097, 2.097]  [-2.097, 2.097]  [-2.097, 2.097]

In [77]:
A

4×4 IntervalMatrix{Float64,Interval{Float64},Array{Interval{Float64},2}}:
     [0, 0]    [0, 0]  [-2500, -2500]    [2500, 2500]
     [0, 0]    [0, 0]          [0, 0]  [-2500, -2500]
   [10, 10]    [0, 0]    [-100, -100]          [0, 0]
 [-10, -10]  [10, 10]          [0, 0]      [-10, -10]

In [78]:
S(10, 10, A)

4×4 IntervalMatrix{Float64,Interval{Float64},Array{Interval{Float64},2}}:
  [5.51581e-08, 5.51582e-08]   …   [4.02887e-07, 4.02888e-07] 
 [-3.80178e-08, -3.80177e-08]     [-1.68779e-07, -1.68778e-07]
  [1.70663e-09, 1.70664e-09]      [-2.86534e-08, -2.86533e-08]
 [-1.61155e-09, -1.61154e-09]      [6.29109e-08, 6.2911e-08]  

In [79]:
exp_overapproximation(A, 1.0, 10) # bloats, it doesn't scale and square

4×4 IntervalMatrix{Float64,Interval{Float64},Array{Interval{Float64},2}}:
 [-8.52379e+120, 8.52379e+120]  …  [-1.18779e+122, 1.18779e+122]
 [-8.52379e+120, 8.52379e+120]     [-1.18779e+122, 1.18779e+122]
 [-8.52379e+120, 8.52379e+120]     [-1.18779e+122, 1.18779e+122]
 [-8.52379e+120, 8.52379e+120]     [-1.18779e+122, 1.18779e+122]

In [84]:
IntervalMatrices.scale_and_square(A, 10, 1.0, 10)

4×4 IntervalMatrix{Float64,Interval{Float64},Array{Interval{Float64},2}}:
  [5.51581e-08, 5.51582e-08]   …   [4.02887e-07, 4.02888e-07] 
 [-3.80178e-08, -3.80177e-08]     [-1.68779e-07, -1.68778e-07]
  [1.70663e-09, 1.70664e-09]      [-2.86534e-08, -2.86533e-08]
 [-1.61155e-09, -1.61154e-09]      [6.29109e-08, 6.2911e-08]  

## Case $\eta = 4$

In [96]:
A = IntervalMatrix(Matrix(tline(η=4)))

8×8 IntervalMatrix{Float64,Interval{Float64},Array{Interval{Float64},2}}:
     [0, 0]      [0, 0]      [0, 0]  …          [0, 0]          [0, 0]
     [0, 0]      [0, 0]      [0, 0]       [2500, 2500]          [0, 0]
     [0, 0]      [0, 0]      [0, 0]     [-2500, -2500]    [2500, 2500]
     [0, 0]      [0, 0]      [0, 0]             [0, 0]  [-2500, -2500]
   [10, 10]      [0, 0]      [0, 0]             [0, 0]          [0, 0]
 [-10, -10]    [10, 10]      [0, 0]  …          [0, 0]          [0, 0]
     [0, 0]  [-10, -10]    [10, 10]         [-10, -10]          [0, 0]
     [0, 0]      [0, 0]  [-10, -10]             [0, 0]      [-10, -10]

In [97]:
S(10, 10, A)

8×8 IntervalMatrix{Float64,Interval{Float64},Array{Interval{Float64},2}}:
  [6.77635e-05, 6.77636e-05]   …   [0.00229669, 0.0022967]    
 [-0.000140969, -0.000140968]     [-0.00332542, -0.00332541]  
  [0.000133764, 0.000133765]       [0.00286481, 0.00286482]   
 [-5.40639e-05, -5.40638e-05]     [-0.00112233, -0.00112232]  
  [5.30467e-06, 5.30468e-06]      [-2.55133e-05, -2.55132e-05]
 [-1.27463e-05, -1.27462e-05]  …   [0.000153128, 0.000153129] 
  [1.42201e-05, 1.42202e-05]      [-0.000199251, -0.00019925] 
 [-9.18678e-06, -9.18676e-06]      [0.000137586, 0.000137587] 

In [98]:
scale_and_square(A, 10, 1.0, 10)

8×8 IntervalMatrix{Float64,Interval{Float64},Array{Interval{Float64},2}}:
  [6.77635e-05, 6.77636e-05]   …   [0.00229669, 0.0022967]    
 [-0.000140969, -0.000140968]     [-0.00332542, -0.00332541]  
  [0.000133764, 0.000133765]       [0.00286481, 0.00286482]   
 [-5.40639e-05, -5.40638e-05]     [-0.00112233, -0.00112232]  
  [5.30467e-06, 5.30468e-06]      [-2.55133e-05, -2.55132e-05]
 [-1.27463e-05, -1.27462e-05]  …   [0.000153128, 0.000153129] 
  [1.42201e-05, 1.42202e-05]      [-0.000199251, -0.00019925] 
 [-9.18678e-06, -9.18676e-06]      [0.000137586, 0.000137587] 